In [1]:
import spacy
from spacy.lang.ru.examples import sentences
from spacy import displacy
from pprint import pprint
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
nlp = spacy.load("ru_core_news_sm")

In [3]:
newsDF = pd.read_csv('news.CSV')
text_news = newsDF['text']
text_news = np.array(text_news)
text_news[2]

'\n                            24 мая исполняется 80 лет поражению Германии в Битве за Атлантику. В этот день гросс-адмирал Дениц прекратил борьбу с конвоями, приказав своим подводникам уйти из Северной Атлантики. Произошло это из-за того, что немецкие «волчьи стаи» несли слишком большие потери. Это стало переломом в подводной войне, шедшей с переменным успехом с 1939 года.\nПоражение неминуемо\nС самого начала Второй мировой войны Германия вела боевые действия против судоходства союзников в Атлантике. Командующий немецким подплавом Карл Дениц в этой битве делал ставку на тактику «волчьей стаи» – атаку конвоя группой лодок. Успехи этот метод стал приносить лишь через год, поскольку поначалу кригсмарине просто не хватало субмарин, чтобы как следует реализовывать эту тактику. Ситуация изменилась осенью 1942-го, когда численность немецких подводных сил в Атлантике выросла до 150 единиц. Тогда-то и началась масштабная охота на конвои.\n"Волчья стая" против конвоя: первый успех "серых волко

In [4]:
doc = nlp(text_news[11])
for s in list(doc.sents)[:2]:
  pprint(list([f'{t.lemma_} ({t.text})' for t in s]))

['\n                             (\n                            )',
 'госдепартамент (Госдепартамент)',
 'сша (США)',
 'одобрить (одобрил)',
 'потенциальный (потенциальную)',
 'продажа (продажу)',
 'норвегия (Норвегии)',
 'шесть (шести)',
 'многоцелевой (многоцелевых)',
 'вертолёт (вертолетов)',
 'mh-60r (MH-60R)',
 ', (,)',
 'об (об)',
 'это (этом)',
 '26 (26)',
 'апрель (апреля)',
 'сообщить (сообщило)',
 'агентство (Агентство)',
 'по (по)',
 'оборонный (оборонному)',
 'сотрудничество (сотрудничеству)',
 'сша (США)',
 '. (.)']
['стоимость (Стоимость)',
 'сделка (сделки)',
 'вместе (вместе)',
 'с (с)',
 'сопутствующий (сопутствующим)',
 'вооружение (вооружением)',
 'оцениваться (оценивается)',
 'в (в)',
 '$ ($)',
 '1 (1)',
 'млрд (млрд)',
 '. (.)',
 '\n (\n)']


In [5]:
for s in list(doc.sents)[:2]:
  displacy.render(s, style="dep", minify=True, jupyter=True, options={"distance":90})

In [6]:
sent0 = list(doc.sents)[0]
childs0 = list(sent0.root.children)
print(sent0.root, childs0)
# Загнать этот вывод в цикл и проанализировать все текста?
print(childs0[0].lemma_, childs0[0].text, childs0[0].dep_, childs0[0].tag_, '->', list(childs0[0].ancestors)[0])

одобрил [Госдепартамент, продажу, сообщило, .]
госдепартамент Госдепартамент nsubj PROPN -> одобрил


In [7]:
with open("rusentilex_2017.txt", encoding="utf-8") as senti_file: 
    for i in range(18): # Пропускаем заголовок файла.
        _ = senti_file.readline()
        
    senti_words = {line.split(", ")[0]: line.split(", ")[3] for line in senti_file.readlines()}

In [8]:
def find_sentiments_spacy(node, senti_words, sentiments):
    for child in node.children:
        if child.lemma_ in senti_words.keys():
            if node.lemma_ not in sentiments.keys():
                sentiments[node.lemma_] = defaultdict(int)
            sentiments[node.lemma_][senti_words[child.lemma_]] += 1

In [9]:
sentiments = {}
for sent in doc.sents:
    find_sentiments_spacy(sent.root, senti_words, sentiments)
sentiments

{}

In [10]:
sentiments = {}
for sent_text in tqdm(text_news):
    doc = nlp(sent_text)
    for sentence in doc.sents:
        find_sentiments_spacy(sentence.root, senti_words, sentiments)

100%|██████████| 1775/1775 [03:51<00:00,  7.66it/s]


In [12]:
[(senti, vals) for senti,vals in sentiments.items() if vals['negative']>30], '\n----\n',\
[(senti, vals) for senti,vals in sentiments.items() if vals['positive']>30]

([('стать',
   defaultdict(int,
               {'positive': 153,
                'пострадать': 1,
                'negative': 100,
                'neutral': 54,
                'positive/negative': 6})),
  ('нанести',
   defaultdict(int,
               {'negative': 32,
                'positive': 3,
                'positive/negative': 3,
                'neutral': 1})),
  ('мочь',
   defaultdict(int,
               {'positive/negative': 17,
                'negative': 57,
                'neutral': 33,
                'positive': 83,
                'пострадать': 1})),
  ('оказаться',
   defaultdict(int,
               {'negative': 54,
                'positive': 51,
                'neutral': 18,
                'positive/negative': 3})),
  ('получить',
   defaultdict(int,
               {'positive': 36,
                'negative': 33,
                'positive/negative': 1,
                'neutral': 8}))],
 '\n----\n',
 [('стать',
   defaultdict(int,
               {'positive': 15